In [2]:
import argparse
import json
import os
import logging
import yaml
from kafka import KafkaProducer
from schema import Schema, SchemaError
log = logging.getLogger()
logging.basicConfig(level=os.environ.get("LOGLEVEL", "INFO"))

In [3]:
def add_value(key):
    match key:
        case 'technology':
            print(key)
            
        case 'hostedAt':
            print(key)

In [4]:
from schema import Schema, SchemaError, Optional, Hook, Or

schema_val = {
    "name": str,
    "description": str,
    "status": str,

    "consumers": {
        "name": str,
        "description": str,
        "type" : str
    },
    "containers": {
        "name": str,
        "sysnonyms": str,
        "description": str,
        Optional("technology", default= lambda : add_value('technology')): str,
        "parentSystem": str,
        "ciDataOwner": str,
        "productOwner": str,
        "applicationType": Or("Business", "Customer Facing", "External Service", "Infrastructure", "Interface", "Office", "Tool", "Unknown"),
        Optional("hostedAt", default = lambda : add_value('hostedAt')): Or("Amazon Web Services (AWS Cloud)", "AT&T", "Azure CF1", "Azure CF2", "Azure Cloud", "DXC", "Equinix", "Google Cloud Platform", "Hybric", "Inlumi", "Local server", "Multi-Cloud", "Not Applicable", "Other", "Salesforce", "ServiceNow", "Solvinity", "Unit4", "Unknown", "User device", "Azure"),
        "deploymentModel": Or("BPO", "CaaS", "IaaS", "On-Premise", "PaaS", "SaaS"),
        "personalData": bool,
        "confidentiality": str,
        "mcv": Or("Highly business critical", "Business critical", "Not business critical", "Not applicable"),
        "maxSeverityLevel": Or(1,2,3,4, "Not applicable"),
        Optional("sox", default= lambda : add_value('sox')): bool,
        Optional("icfr", default= lambda : add_value('icfr')): bool,
        "assignementGroup": str,
        "operationalStatus": Or("Pipelined", "Operational", "Non-Operational", "Submitted for decommissioning", "Decommissioned", "In decommissioning process"),
        "environments": Or("nl", "be"),
        "relationships": {
            "type": str,
            "container": {
                "name": str,
            },
        },
        "components": {
            "name": str,
            "description": str,
            "exposedAPIs": {
                "name": str,
                "description": str,
                "type": str,
                "status": str,
            },
            "consumedAPIs": {
                "name": str,
                "description": str,
                "status": str
            }
        },
    }
}

In [5]:
def validate_yaml(yaml_data):
    #schema = eval(open('./schema.yml', 'r').read())
    validator = Schema(schema_val)
    try:
        validator.validate(yaml_data)
        print('YML valid')
    except SchemaError as se:
        print(se)

In [6]:
def load_doc():
    with open('./test.yml', 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as e:
            print(e)

In [7]:
doc = load_doc()
validate_yaml(doc)

YML valid


In [12]:
schema = {
  "type" : "record",
  "name" : "SystemModel",
  "namespace" : "com.test.avro",
  "fields" : [ {
    "name" : "name",
    "type" : "string"
  }, {
    "name" : "description",
    "type" : "string"
  }, {
    "name" : "status",
    "type" : "string"
  }, {
    "name" : "consumers",
    "type" : {
      "type" : "record",
      "name" : "consumers",
      "fields" : [ {
        "name" : "name",
        "type" : "string"
      }, {
        "name" : "description",
        "type" : "string"
      }, {
        "name" : "type",
        "type" : "string"
      } ]
    }
  }, {
    "name" : "containers",
    "type" : {
      "type" : "record",
      "name" : "containers",
      "fields" : [ {
        "name" : "name",
        "type" : "string"
      }, {
        "name" : "sysnonyms",
        "type" : "string"
      }, {
        "name" : "description",
        "type" : "string"
      }, {
        "name" : "technology",
        "type" : "string"
      }, {
        "name" : "parentSystem",
        "type" : "string"
      }, {
        "name" : "ciDataOwner",
        "type" : "string"
      }, {
        "name" : "productOwner",
        "type" : "string"
      }, {
        "name" : "applicationType",
        "type" : "string"
      }, {
        "name" : "hostedAt",
        "type" : "string"
      }, {
        "name" : "deploymentModel",
        "type" : "string"
      }, {
        "name" : "personalData",
        "type" : "boolean"
      }, {
        "name" : "confidentiality",
        "type" : "string"
      }, {
        "name" : "mcv",
        "type" : "string"
      }, {
        "name" : "maxSeverityLevel",
        "type" : "long"
      }, {
        "name" : "sox",
        "type" : "boolean"
      }, {
        "name" : "icfr",
        "type" : "boolean"
      }, {
        "name" : "assignementGroup",
        "type" : "string"
      }, {
        "name" : "operationalStatus",
        "type" : "string"
      }, {
        "name" : "environments",
        "type" : "string"
      }, {
        "name" : "relationships",
        "type" : {
          "type" : "record",
          "name" : "relationships",
          "fields" : [ {
            "name" : "type",
            "type" : "string"
          }, {
            "name" : "container",
            "type" : {
              "type" : "record",
              "name" : "container",
              "fields" : [ {
                "name" : "name",
                "type" : "string"
              } ]
            }
          } ]
        }
      }, {
        "name" : "components",
        "type" : {
          "type" : "record",
          "name" : "components",
          "fields" : [ {
            "name" : "name",
            "type" : "string"
          }, {
            "name" : "description",
            "type" : "string"
          }, {
            "name" : "exposedAPIs",
            "type" : {
              "type" : "record",
              "name" : "exposedAPIs",
              "fields" : [ {
                "name" : "name",
                "type" : "string"
              }, {
                "name" : "description",
                "type" : "string"
              }, {
                "name" : "type",
                "type" : "string"
              }, {
                "name" : "status",
                "type" : "string"
              } ]
            }
          }, {
            "name" : "consumedAPIs",
            "type" : {
              "type" : "record",
              "name" : "consumedAPIs",
              "fields" : [ {
                "name" : "name",
                "type" : "string"
              }, {
                "name" : "description",
                "type" : "string"
              }, {
                "name" : "status",
                "type" : "string"
              } ]
            }
          } ]
        }
      } ]
    }
  } ]
}

In [20]:
data = load_doc()

In [21]:
from kafka_schema_registry import prepare_producer

producer = prepare_producer(bootstrap_servers=["10.152.183.181:9094"], avro_schema_registry="http://10.152.183.242:8081", topic_name="topic1", value_schema=schema, num_partitions=1, replication_factor=1)

producer.send("topic1",data)

INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=10.152.183.181:9094 <connecting> [IPv4 ('10.152.183.181', 9094)]>: connecting to 10.152.183.181:9094 [('10.152.183.181', 9094) IPv4]
INFO:kafka.conn:Probing node bootstrap-0 broker version
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=10.152.183.181:9094 <connecting> [IPv4 ('10.152.183.181', 9094)]>: Connection complete.
INFO:kafka.conn:Broker version identified as 2.5.0
INFO:kafka.conn:Set configuration api_version=(2, 5, 0) to skip auto check_version requests on startup
INFO:kafka.conn:Probing node bootstrap-0 broker version
INFO:kafka.conn:Broker version identified as 2.5.0
INFO:kafka.conn:Set configuration api_version=(2, 5, 0) to skip auto check_version requests on startup
INFO:kafka.conn:<BrokerConnection node_id=0 host=192.168.0.121:30000 <connecting> [IPv4 ('192.168.0.121', 30000)]>: connecting to 192.168.0.121:30000 [('192.168.0.121', 30000) IPv4]
INFO:kafka.conn:Probing node 0 broker version
INFO:kafka.co

INFO:kafka.conn:<BrokerConnection node_id=0 host=192.168.0.121:30000 <connecting> [IPv4 ('192.168.0.121', 30000)]>: connecting to 192.168.0.121:30000 [('192.168.0.121', 30000) IPv4]
INFO:kafka.conn:<BrokerConnection node_id=0 host=192.168.0.121:30000 <connecting> [IPv4 ('192.168.0.121', 30000)]>: Connection complete.
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=10.152.183.181:9094 <connected> [IPv4 ('10.152.183.181', 9094)]>: Closing connection. 
INFO:kafka.conn:<BrokerConnection node_id=0 host=192.168.0.121:30000 <connecting> [IPv4 ('192.168.0.121', 30000)]>: connecting to 192.168.0.121:30000 [('192.168.0.121', 30000) IPv4]
INFO:kafka.conn:<BrokerConnection node_id=0 host=192.168.0.121:30000 <connecting> [IPv4 ('192.168.0.121', 30000)]>: Connection complete.
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=10.152.183.181:9094 <connected> [IPv4 ('10.152.183.181', 9094)]>: Closing connection. 


In [4]:
import os
import sys

import requests

schema_registry_url = "http://10.152.183.181:9094"
topic = "topic1"
schema_file = "order.json"

aboslute_path_to_schema = os.path.join(os.getcwd(), schema_file)

print("Schema Registry URL: " + schema_registry_url)
print("Topic: " + topic)
print("Schema file: " + schema_file)
print

with open(aboslute_path_to_schema, 'r') as content_file:
    schema = content_file.read()

payload = "{ \"schema\": \"" \
          + schema.replace("\"", "\\\"").replace("\t", "").replace("\n", "") \
          + "\" }"

url = schema_registry_url + "/subjects/" + topic + "-value/versions"
headers = {"Content-Type": "application/vnd.schemaregistry.v1+json"}

r = requests.post(url, headers=headers, data=payload)
if r.status_code == requests.codes.ok:
    print("Success")
else:
    r.raise_for_status()

Schema Registry URL: http://10.152.183.181:9094
Topic: topic1
Schema file: order.json


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))